# NOTE: This document is outdated and will be updated soon. Please refer to the latest Python SDK documentation [here](https://developers.deepgram.com/docs/python-sdk) for the most up-to-date info moving forward.

# Transcribe any audio file with Deepgram!

**Make a copy of this notebook into your own drive, and follow the instructions below!** 🥳🥳🥳


----------------------------

# Get started:

1) Copy this notebook (`File > Save a copy in Drive`) or download the .ipynb (`File > Download > Download as .ipynb`).

2) Follow the instructions below!

----------------------------
#Instructions:

Running the following three cells will allow you to transcribe any audio you wish. The comments below point out the variables you can manipulate to modify your output as you wish.

Before running this notebook, you'll need to have a couple audio files on-hand
that you wish to transcribe. Once you have those files in a folder, you should be able to transcribe as you please. Just specify the filepaths as outlined below!

And by the way, if you haven't yet signed up for Deepgram, check out this link here: https://dpgr.am/prerecorded-notebook-signup

# Step 1: Dependencies

Run this cell to download all necessary dependencies.

Note: You can run a cell by clicking the play button on the left or by clicking on the cell and pressing `shift`+`ENTER` at the same time. (Or `shift` + `return` on Mac).

# NOTE: This document is outdated and will be updated soon. Please refer to the latest Python SDK documentation [here](https://developers.deepgram.com/docs/python-sdk) for the most up-to-date info moving forward.

In [1]:
!pip install deepgram-sdk==3.*
!pip install dataclasses typing-extensions verboselogs
!pip install python-dotenv
!pip install num2words
!pip install vaderSentiment

  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
  Using cached num2words-0.5.13-py3-none-any.whl (143 kB)
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=22a2d96631d1ce9a054fd1797ab8e4409d211c8bec3bc722dfa74c126fa42294
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.1 MB/s eta 0:00:00


# Step 2: Upload audio files to this Colab!

On the left, you'll see a side-bar with a folder icon. Click that icon, and you'll see a series of folders. This is where you'll upload your audios.

You can upload your files directly into this directory by clicking the upload icon in the top left. The icon looks like a sheet of paper with an upwards-pointing arrow on it.

Click the upload icon and select the audio file you wish to transcribe. It will take a few moments for the audio to appear, but once it does, move onto Step 3.

# NOTE: This document is outdated and will be updated soon. Please refer to the latest Python SDK documentation [here](https://developers.deepgram.com/docs/python-sdk) for the most up-to-date info moving forward.

In [2]:
# Have you completed Step 2 above? 👀
# Do you see your audio file in the folder on the left? 📂

# Step 3: Transcription

Fill in the following variables:


* `dg_key` = Your personal Deepgram API key
* `MIMETYPE` = the type of audio file you're working with (mp3, mp4, m4a, etc.)
* `DIRECTORY` = The name of the folder that contains the audio(s) you wish to transcribe. Note, unless you created a new folder for your audios, the default `'.'` value should be fine. (Or, if you placed your audio file in the default `sample_data` folder, set the value of `DIRECTORY` to `sample_data`.)


Now run the cell! (`Shift` + `Enter`)

-----------



And by the way, if you're already a Deepgram user, and you're getting an error in this cell the most common fixes are:

1. You may need to update your installation of the deepgram-sdk.
2. You may need to check how many credits you have left in your Deepgram account.


# NOTE: This document is outdated and will be updated soon. Please refer to the latest Python SDK documentation [here](https://developers.deepgram.com/docs/python-sdk) for the most up-to-date info moving forward.

In [3]:
# Copyright 2023-2024 Deepgram SDK contributors. All Rights Reserved.
# Use of this source code is governed by a MIT license that can be found in the LICENSE file.
# SPDX-License-Identifier: MIT


import os
import httpx
import json
import re
import logging, verboselogs
import pandas as pd
import nltk
from dotenv import load_dotenv
from datetime import datetime
from num2words import num2words
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from deepgram import (
    DeepgramClient,
    DeepgramClientOptions,
    PrerecordedOptions,
    FileSource,
    )


# The list of common words such as "a", "an", etc.
nltk.download('stopwords')
# Download a collection of popular resources from the NLTK library
nltk.download('popular', quiet=True)
# Sentence tokenization (Splitting a text into individual senteces)
nltk.download('punkt')
# English vocabulary database
nltk.download('wordnet')
# Used for training language models or evaluating nlp algothms
nltk.download('brown')
# Used for performing sentiment analysis on text
nltk.download('vader_lexicon')


lemmatizer = WordNetLemmatizer()
analyzer = SentimentIntensityAnalyzer()
stemmer = SnowballStemmer("english")
stop_words = stopwords.words("english")


load_dotenv()



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


False

In [4]:
def extract_transcript(AUDIO_FILE):
    try:
        # STEP 1 Create a Deepgram client using the API key in the environment variables
        config: DeepgramClientOptions = DeepgramClientOptions(
            verbose=logging.SPAM,
        )
        deepgram: DeepgramClient = DeepgramClient("03feccffc4af0aac76e14882c4b7e5b08b509b92", config)
        # OR use defaults
        # deepgram: DeepgramClient = DeepgramClient()

        # STEP 2 Call the transcribe_file method on the prerecorded class
        with open(AUDIO_FILE, "rb") as file:
            buffer_data = file.read()

        payload: FileSource = {
            "buffer": buffer_data,
        }

        options: PrerecordedOptions = PrerecordedOptions(
            model="nova",
            smart_format=True,
            utterances=True,
            punctuate=True,
            diarize=False,
            numerals=False,
        )

        before = datetime.now()
        response = deepgram.listen.prerecorded.v("1").transcribe_file(
            payload, options, timeout=httpx.Timeout(300.0, connect=10.0)
        )
        after = datetime.now()

        print(response.to_json(indent=4))
        audio_dir = os.path.dirname(AUDIO_FILE)

        # Construct the path for the JSON file in the same directory as the audio file
        json_filename = os.path.join(audio_dir, "transcript.json")

        # Write the JSON data to the file in the same directory as the audio file
        with open(json_filename, "w") as json_file:
            json_file.write(response.to_json(indent=4))

        print(f"Transcript JSON file '{json_filename}' created successfully.")
        return json_filename

    except Exception as e:
        print(f"Exception: {e}")

In [5]:
# Extracting the transcript from the json file
# and saving it into a list
def json_to_list(transcription_file):
  sentences = []
  with open(transcription_file, "r") as file:
        data = json.load(file)
        result = data['results']['channels'][0]['alternatives'][0]['transcript']
        result = result.split('.')
        for sentence in result:
          sentences.append(sentence)
        return sentences


In [6]:
def clean(list):
  cleaned = []
  for i in list:
    sentence = sent_tokenize(i)
    for item in sentence:
      # Convert to lowercase
      text_lowercase = item.lower()
      # Remove punctuation
      text_without_punctuation = re.sub(r"[^\w\s]", "", text_lowercase)
      # Remove stopwords and stem words
      tokens = word_tokenize(text_without_punctuation)
      new_tokens = []
      for word in tokens:
        if word.isnumeric():
          word = num2words(word)
          new_tokens.append(lemmatizer.lemmatize(word))
          continue
        elif word not in stop_words:
          new_tokens.append(lemmatizer.lemmatize(word))
          continue
      # Join tokens back into a string
      cleaned_text = " ".join(new_tokens)
      cleaned.append(cleaned_text)
  return cleaned

In [7]:
# Use Vader library to get the polarity of the sentence
# Sentences with polarity more than zero are positive, whilst sentences with polarity less than zero are negative
# Sentences with zero polarity are neutral

def getPolarity(text):
    polarity = SentimentIntensityAnalyzer().polarity_scores(text)
    if polarity['compound'] > 0:
      sentiment = 'postive'
    elif polarity['compound'] < 0:
      sentiment = 'negative'
    else:
      sentiment = 'neutral'
    return sentiment,polarity

In [9]:
# https://youtu.be/Ls3uQD6eQpM?si=O37fev3MdbGrDGCO

audio_file1 = "/content/Ep_91_Accenture_to_Make_2_4B_from_Gen_AI_AI_Industry_s_Ls3uQD6eQpM.m4a"

json_transcript1 = extract_transcript(audio_file1)

pod1 = json_to_list(json_transcript1)

cleaned1 = clean(pod1)

cleaned1

Version.v ENTER
Version.v ENTER
DEBUG:deepgram.clients.listen:Version.v ENTER
version: 1
version: 1
INFO:deepgram.clients.listen:version: 1
path: deepgram.clients.prerecorded.v1.client
path: deepgram.clients.prerecorded.v1.client
INFO:deepgram.clients.listen:path: deepgram.clients.prerecorded.v1.client
className: PreRecordedClient
className: PreRecordedClient
INFO:deepgram.clients.listen:className: PreRecordedClient
Version.v succeeded
Version.v succeeded
NOTICE:deepgram.clients.listen:Version.v succeeded
Version.v LEAVE
Version.v LEAVE
DEBUG:deepgram.clients.listen:Version.v LEAVE
PreRecordedClient.transcribe_file ENTER
DEBUG:deepgram.clients.prerecorded.v1.client:PreRecordedClient.transcribe_file ENTER
url: https://api.deepgram.com/v1/listen
INFO:deepgram.clients.prerecorded.v1.client:url: https://api.deepgram.com/v1/listen
PrerecordedOptions switching class -> json
INFO:deepgram.clients.prerecorded.v1.client:PrerecordedOptions switching class -> json
options: {'diarize': False, 'mod

Transcript JSON file '/content/transcript.json' created successfully.


['still believe directionally true people best chance survive thrive job loss come industry company people ai literate high performing people always true',
 'welcome artificial intelligence show podcast help grow smarter making ai approachable actionable',
 'name paul racery',
 'im founder ceo marketing institute im host',
 'week im joined co host marketing ai institute chief content officer mike caput break ai news matter give insight perspective use advance company career',
 'join u accelerate lintelligent show',
 'im host paul ratzer along co host always mike caput',
 'hey mike',
 'hey paul',
 '',
 'eclipse day',
 '',
 'recording monday april 8th one thousand and fifteen',
 'awesome weekend cleveland',
 'daughter actually went woman final four championship yesterday really cool get take',
 'shes twelve love basketball',
 'cool experience get',
 'cleveland like center world today',
 'every airbnb sold',
 'every hotel sold',
 'people watch clip like prime range u see',
 'waiting cloud

In [11]:
# https://www.youtube.com/watch?v=0ZESX1BOgL4

audio_file2 = "/content/The_Future_of_Artificial_Intelligence_Animated_0ZESX1BOgL4_140.mp3"

json_transcript2 = extract_transcript(audio_file2)

pod2 = json_to_list(json_transcript2)

cleaned2 = clean(pod2)

cleaned2

Streaming output truncated to the last 5000 lines.
                        "confidence": 0.8828125,
                        "punctuated_word": "creators"
                    },
                    {
                        "word": "adhere",
                        "start": 432.455,
                        "end": 432.775,
                        "confidence": 0.9980469,
                        "punctuated_word": "adhere"
                    },
                    {
                        "word": "to",
                        "start": 432.775,
                        "end": 432.935,
                        "confidence": 0.9995117,
                        "punctuated_word": "to"
                    },
                    {
                        "word": "a",
                        "start": 432.935,
                        "end": 433.095,
                        "confidence": 0.9995117,
                        "punctuated_word": "a"
                    },
                    {
         

Streaming output truncated to the last 5000 lines.
                        "word": "all",
                        "start": 430.85498,
                        "end": 431.255,
                        "confidence": 0.99902344,
                        "punctuated_word": "all"
                    },
                    {
                        "word": "ai",
                        "start": 431.255,
                        "end": 431.655,
                        "confidence": 0.9067383,
                        "punctuated_word": "AI"
                    },
                    {
                        "word": "creators",
                        "start": 431.655,
                        "end": 432.155,
                        "confidence": 0.8828125,
                        "punctuated_word": "creators"
                    },
                    {
                        "word": "adhere",
                        "start": 432.455,
                        "end": 432.775,
                      

['world today artificial intelligence one exciting rapidly advancing technological field',
 'recent exploration deep learning enabled computer think like human opening door critical field science technology',
 'powerful intelligent machine design complex experiment interpret large quantity data much faster human researcher',
 'deep learning system like deep mind alpha fold accelerate slow moving research field like biology chemistry',
 'thus expanding understanding life universe',
 'time content generating system called generative transformed work socialize interact world',
 'program use deep learning pro identify pattern within training data apply pattern create original content',
 'sophisticated image generator produce photorealistic image simple text allowing anyone credit card bring artistic vision life',
 'perhaps impressive large foundation model like gpt solve complex problem use natural language outperform human variety specific task',
 'currently artificial intelligence evolvi

In [12]:
# https://www.youtube.com/watch?v=eXdVDhOGqoE

audio_file3 = "/content/AI_Is_Dangerous_but_Not_for_the_Reasons_You_Think_Sasha_eXdVDhOGqoE.mp3"

json_transcript3 = extract_transcript(audio_file3)

pod3 = json_to_list(json_transcript3)

cleaned3 = clean(pod3)

cleaned3

Streaming output truncated to the last 5000 lines.
                        "confidence": 0.99072266,
                        "punctuated_word": "experienced"
                    },
                    {
                        "word": "this",
                        "start": 376.69,
                        "end": 376.93,
                        "confidence": 0.9970703,
                        "punctuated_word": "this"
                    },
                    {
                        "word": "firsthand",
                        "start": 376.93,
                        "end": 377.43,
                        "confidence": 0.9951172,
                        "punctuated_word": "firsthand"
                    },
                    {
                        "word": "when",
                        "start": 377.65,
                        "end": 377.88998,
                        "confidence": 0.96191406,
                        "punctuated_word": "when"
                    },
             

Streaming output truncated to the last 5000 lines.
                        "start": 375.72998,
                        "end": 376.22998,
                        "confidence": 0.67633057,
                        "punctuated_word": "Bolomini,"
                    },
                    {
                        "word": "experienced",
                        "start": 376.28998,
                        "end": 376.69,
                        "confidence": 0.99072266,
                        "punctuated_word": "experienced"
                    },
                    {
                        "word": "this",
                        "start": 376.69,
                        "end": 376.93,
                        "confidence": 0.9970703,
                        "punctuated_word": "this"
                    },
                    {
                        "word": "firsthand",
                        "start": 376.93,
                        "end": 377.43,
                        "confidence": 0.99

['ive ai researcher decade',
 'couple month ago got weirdest email career',
 'random stranger wrote saying work ai gon na end humanity',
 'get',
 'ai hot right',
 'line pretty much every day sometimes really cool thing like discovering new molecule medicine dope pope white puffer coat',
 'time headline really dark',
 'like chatbot telling guy divorce wife ai meal planner app proposing crowd pleasing recipe featuring chlorine gas',
 'background weve heard lot talk doomsday scenario existential risk singularity letter written event organized make sure doesnt happen',
 'im researcher study ai impact society',
 'dont know whats gon na happen ten twenty year nobody really',
 'know there pretty nasty thing going right ai doesnt exist vacuum',
 'part society impact people planet',
 'ai model contribute climate change training data us art book created artist author without consent deployment discriminate entire community',
 'need start tracking impact',
 'need start transparent disclosing crea

In [13]:
# https://www.youtube.com/watch?v=zu_pyp4Jsi0

audio_file4 = "/content/The_Future_of_Generative_AI_ft_Anthropic_Cohere_Salesf_zu_pyp4Jsi0.mp3"

json_transcript4 = extract_transcript(audio_file4)

pod4 = json_to_list(json_transcript4)

cleaned4 = clean(pod4)

cleaned4

Streaming output truncated to the last 5000 lines.
                        "end": 766.48505,
                        "confidence": 0.7429199,
                        "punctuated_word": "x,"
                    },
                    {
                        "word": "y",
                        "start": 766.48505,
                        "end": 766.565,
                        "confidence": 0.99658203,
                        "punctuated_word": "y,"
                    },
                    {
                        "word": "and",
                        "start": 766.565,
                        "end": 766.72504,
                        "confidence": 0.9926758,
                        "punctuated_word": "and"
                    },
                    {
                        "word": "z",
                        "start": 766.72504,
                        "end": 766.885,
                        "confidence": 0.6810303,
                        "punctuated_word": "z,"
                 

Transcript JSON file '/content/transcript.json' created successfully.


['great back',
 'im clara shai ceo service cloud',
 'im trailblazer dx event san francisco martin khan coher dario amadei anthropic',
 'welcome congratulation salesforce venture generative ai fund investment announced yesterday',
 'welcome ecosystem community',
 'thank',
 'thank clara',
 'question',
 'know announced einstein gpt yesterday combining proprietary salesforce ai model ecosystem vetted generative ai partner company developed leading solution market',
 'know could share important consideration youve built foundational model thinking differentiate offering space',
 'dario maybe wan na start go martin',
 'yeah',
 'think side u builder side developer developing top side customer thing around reliability safety trust often paramount',
 'point know model world seen capable concern know predictable',
 'people expect',
 'different area',
 'know one know there problem hallucination know model sometimes make thing whole cloth',
 'one thing weve worked lot reducing problem making model

In [14]:
# https://www.youtube.com/watch?v=mXXzhQhe6t8

audio_file5 = "/content/Snowflake_s_Torsten_Grabs_Offers_His_Thoughts_On_The_Fu_mXXzhQhe6t8.mp3"

json_transcript5 = extract_transcript(audio_file5)

pod5 = json_to_list(json_transcript5)

cleaned5 = clean(pod5)

cleaned5

Streaming output truncated to the last 5000 lines.
                        "confidence": 0.9995117,
                        "punctuated_word": "a"
                    },
                    {
                        "word": "much",
                        "start": 235.425,
                        "end": 235.74501,
                        "confidence": 1.0,
                        "punctuated_word": "much"
                    },
                    {
                        "word": "more",
                        "start": 235.74501,
                        "end": 235.905,
                        "confidence": 0.9995117,
                        "punctuated_word": "more"
                    },
                    {
                        "word": "conversational",
                        "start": 235.905,
                        "end": 236.405,
                        "confidence": 1.0,
                        "punctuated_word": "conversational"
                    }
                ],
  

Streaming output truncated to the last 5000 lines.
                        "punctuated_word": "to"
                    },
                    {
                        "word": "have",
                        "start": 235.10501,
                        "end": 235.265,
                        "confidence": 1.0,
                        "punctuated_word": "have"
                    },
                    {
                        "word": "a",
                        "start": 235.265,
                        "end": 235.425,
                        "confidence": 0.9995117,
                        "punctuated_word": "a"
                    },
                    {
                        "word": "much",
                        "start": 235.425,
                        "end": 235.74501,
                        "confidence": 1.0,
                        "punctuated_word": "much"
                    },
                    {
                        "word": "more",
                        "start":

['set stage audience might familiar generative ai llm provide quick overview trend youre noticing collectively',
 'yeah',
 'think really interesting moment industry user technology computer generally',
 'opportunity right loms generative ai fundamentally interaction human computer technology change dramatic way',
 'past computer prescriptive user interact get result wanted',
 'right',
 'actually change ai large language model much conversational experience',
 'start interact system natural language still get result system',
 'thats huge opportunity u make technology much approachable much much wider audience',
 'know torsten seeing rapid increase adoption mean future industry',
 'yeah',
 'think two reason seeing interest adoption',
 'first one come back conversational experience user computer technology empowering user become much productive',
 'imagine developer write code build application',
 'co pilot driven generative ai help build application write code literally save ninety time 

In [15]:
# after adding all the podcasts

result = []

result.extend(cleaned1 + cleaned2 + cleaned3 + cleaned4 + cleaned5)

print (result)


['still believe directionally true people best chance survive thrive job loss come industry company people ai literate high performing people always true', 'welcome artificial intelligence show podcast help grow smarter making ai approachable actionable', 'name paul racery', 'im founder ceo marketing institute im host', 'week im joined co host marketing ai institute chief content officer mike caput break ai news matter give insight perspective use advance company career', 'join u accelerate lintelligent show', 'im host paul ratzer along co host always mike caput', 'hey mike', 'hey paul', '', 'eclipse day', '', 'recording monday april 8th one thousand and fifteen', 'awesome weekend cleveland', 'daughter actually went woman final four championship yesterday really cool get take', 'shes twelve love basketball', 'cool experience get', 'cleveland like center world today', 'every airbnb sold', 'every hotel sold', 'people watch clip like prime range u see', 'waiting cloud go away hope everyon

In [18]:
data = []  # List to store dictionaries
for sentence in result:
  sentiment, polarity = getPolarity(sentence)
  new_item = {'sentence' : sentence, 'sentiment' : sentiment, 'category' : 'GenAI'}    # add category here
  data.append(new_item)

df = pd.DataFrame(data)

In [19]:
# Make sure to change category
df.to_csv('GenAI data (podcast-scraped).csv', index=False)
